Для выполнения всех следующих задач вам необходимо выбрать два текстовых файла примерно одинаковой длины и похожего жанра на двух разных языках, будет неплохо, если это окажется один и тот же текст в переводе.

#### Задача 1.

Просмотрите оба выбранных текста. Удостоверьтесь, что тексты чистые, если же в них есть какой-то мусор: хештеги, затесавшиеся при OCR символы и подобное, почистите с помощью регулярных выражений.

Проведите первичный статистический анализ: разбейте тексты на предложения и на токены, посчитайте количество того и другого, сопоставьте. Какой текст оказался длиннее? В каком тексте средняя длина предложения больше? Почему? В каком тексте выше лексическое разнообразие?

Таким образом, вам необходимо узнать следующие вещи:

- количество предложений
- количество токенов
- средняя длина предложения (среднее количество слов в предложении)
- соотношение "уникальные токены / все токены"

In [ ]:
!pip install stanza

In [2]:
import stanza

In [6]:
with open('/content/Dog_Star-Arthur_Clarke.txt', encoding='utf8') as f:
    eng_text = f.read()

with open('/content/Кларк Артур. Созвездие Пса.txt', encoding='windows-1251') as f:
    rus_text = f.read()

In [ ]:
nlp_ru = stanza.Pipeline(lang='ru', processors='tokenize,pos,lemma,depparse')
nlp_en = stanza.Pipeline(lang='en', processors='tokenize,pos,lemma,depparse')

In [17]:
def count_statistics(text):

    if text == eng_text:
        my_nlp = nlp_en
    else:
        my_nlp = nlp_ru

    doc = my_nlp(text)

    sentences_number = len(doc.sentences)
    print(f'Кол-во предложений: {sentences_number}')

    tokens_number = sum(len(sentence.tokens) for sentence in doc.sentences)
    print(f'Кол-во токенов: {tokens_number}')

    avg_sentence_length = tokens_number / sentences_number
    print(f'Средняя длина предложения: {avg_sentence_length:.2f}')

    # список уникальных токенов
    unique_tokens = set([token.text for sentence in doc.sentences for token in sentence.tokens])
    unique_all = len(unique_tokens) / tokens_number
    print(f'Соотношение "уникальные токены / все токены": {unique_all:.2f}')

In [18]:
count_statistics(eng_text)

Кол-во предложений: 89
Кол-во токенов: 1837
Средняя длина предложения: 20.64
Соотношение "уникальные токены / все токены": 0.35


In [19]:
count_statistics(rus_text)

Кол-во предложений: 133
Кол-во токенов: 2140
Средняя длина предложения: 16.09
Соотношение "уникальные токены / все токены": 0.48


In [20]:
# Текст на русском длиннее.
# Средняя длина предложения больше в тексте на английском. Видимо, используются более сложные предложения - они длиннее, но их меньше.
# Лексическое разнообразие выше в русском тексте.

#### Задача 2 (4 балла).

Сделайте разборы ваших текстов в формате UD. Посчитайте, какой процент токенов по частям речи имеет совпадающие со словоформой леммы (т.е., в скольких случаях токены с частью речи VERB, например, имели словарную форму: и сам токен, и лемма одинаковые). Что вы можете сказать о выбранных вами языках на основании этих данных? Ожидаются две таблички с процентами совпадающих по лемме и токену слов для каждой части речи.

In [ ]:
!pip install prettytable

In [22]:
from prettytable import PrettyTable

In [65]:
def token_is_lemma(text):

    if text == eng_text:
        my_nlp = nlp_en
    else:
        my_nlp = nlp_ru

    doc = my_nlp(text)

    d = {} # словарик типа {часть речи:кол-во совпадений леммы и словоформы}
    for sentence in doc.sentences:
        for word in sentence.words:
            if word.text == word.lemma:
                if word.upos in d.keys():
                    d[word.upos] += 1
                else:
                    d[word.upos] = 1

    tokens_number = sum(len(sentence.tokens) for sentence in doc.sentences)

    # рисуем табличку
    table = PrettyTable()
    table.field_names = ["POS", "percent"]
    for k, v in d.items():
        percentage = v/tokens_number * 100
        table.add_row([k, f'{round(percentage, 2)}%'])

    return table

In [66]:
print(token_is_lemma(eng_text))

+-------+---------+
|  POS  | percent |
+-------+---------+
|  PRON |  10.51% |
| PROPN |   2.5%  |
|  PART |  2.12%  |
|  ADJ  |  6.15%  |
|  NOUN |  9.85%  |
| PUNCT |  11.27% |
|  ADP  |  9.64%  |
| CCONJ |  2.78%  |
|  ADV  |   6.1%  |
|  DET  |  7.62%  |
|  VERB |  3.97%  |
| SCONJ |  2.29%  |
|  AUX  |  2.07%  |
|  NUM  |  0.71%  |
|  INTJ |  0.05%  |
|  SYM  |  0.11%  |
+-------+---------+


In [68]:
print(token_is_lemma(rus_text))

+-------+---------+
|  POS  | percent |
+-------+---------+
|  NOUN |  5.14%  |
|  ADP  |   6.4%  |
|  ADJ  |  1.07%  |
|  PART |   4.3%  |
| PUNCT |  18.5%  |
| CCONJ |   2.9%  |
|  ADV  |  5.65%  |
|  PRON |  4.91%  |
|  VERB |  2.85%  |
| SCONJ |  2.62%  |
|  NUM  |  0.65%  |
|  DET  |  0.56%  |
| PROPN |  0.75%  |
|  AUX  |  0.14%  |
+-------+---------+


In [ ]:
# выводы логичные))

# в английском чаще всего лемма и словоформа сопадают у неизменяемых частей речи
# и у местоимений с сущ - в русском это в два раза реже из-за падежей

# в русском вообще маленький разброс по процентам среди частей речи, т.к. почти все как-то да изменяется
# чаще всего совпадают у неизменяемых частей речи

# у глаголов в обоих языках маленький процент и разброс небольшой

#### Задача 3 (2 балла).

Посчитайте медианную длину предложения для ваших текстов (медиана - это если взять все длины всех ваших предложений, упорядочить их от маленького к большому и выбрать то число, которое оказалось посередине, а если чисел четное количество, то взять среднее арифметическое двух чисел посередине. Например, если у вас пять предложений длинами 1, 2, 6, 7, 8, то медиана - 6, а если шесть предложений длинами 1, 1, 7, 9, 10, 11, то медиана - (7 + 9) / 2 = 8). Возьмите любые два предложения (на разных языках) и постройте для них деревья зависимостей. Изучите связи зависимостей (deprel) и вершины: согласны ли вы с разбором?

In [90]:
def median_length(text):

    if text == eng_text:
        my_nlp = nlp_en
    else:
        my_nlp = nlp_ru

    doc = my_nlp(text)

    lengths = [] # здесь будем собирать длины предложений
    for sentence in doc.sentences:
        lengths.append(len(doc.sentences[0].tokens))

    # сортируем длины
    lengths.sort()

    # вычисляем медиану
    n = len(lengths)
    if n % 2 == 0:
        median = (lengths[n // 2 - 1] + lengths[n // 2]) / 2
    else:
        median = lengths[n // 2]

    return median

In [91]:
median_length(eng_text)

13

In [92]:
median_length(rus_text)

9

In [98]:
def dependency_tree(sentence):

    if sentence[1] == 'ru':
        doc = nlp_ru(sentence[0])
    else:
        doc = nlp_en(sentence[0])

    print(*[f'id: {word.id}\tword: {word.text}\thead id: {word.head}\thead: {sent.words[word.head-1].text if word.head > 0 else "root"}\tdeprel: {word.deprel}' for sent in doc.sentences for word in sent.words], sep='\n')


In [99]:
dependency_tree(['She was a beautiful animal, about 95% Alsatian.', 'eng'])

id: 1	word: She	head id: 5	head: animal	deprel: nsubj
id: 2	word: was	head id: 5	head: animal	deprel: cop
id: 3	word: a	head id: 5	head: animal	deprel: det
id: 4	word: beautiful	head id: 5	head: animal	deprel: amod
id: 5	word: animal	head id: 0	head: root	deprel: root
id: 6	word: ,	head id: 10	head: Alsatian	deprel: punct
id: 7	word: about	head id: 8	head: 95	deprel: advmod
id: 8	word: 95	head id: 9	head: %	deprel: nummod
id: 9	word: %	head id: 10	head: Alsatian	deprel: compound
id: 10	word: Alsatian	head id: 5	head: animal	deprel: parataxis
id: 11	word: .	head id: 5	head: animal	deprel: punct


In [100]:
dependency_tree(['Это было великолепное животное, почти чистокровная восточноевропейская овчарка.', 'ru'])

id: 1	word: Это	head id: 4	head: животное	deprel: nsubj
id: 2	word: было	head id: 4	head: животное	deprel: cop
id: 3	word: великолепное	head id: 4	head: животное	deprel: amod
id: 4	word: животное	head id: 0	head: root	deprel: root
id: 5	word: ,	head id: 9	head: овчарка	deprel: punct
id: 6	word: почти	head id: 7	head: чистокровная	deprel: obl
id: 7	word: чистокровная	head id: 9	head: овчарка	deprel: amod
id: 8	word: восточноевропейская	head id: 9	head: овчарка	deprel: amod
id: 9	word: овчарка	head id: 4	head: животное	deprel: conj
id: 10	word: .	head id: 4	head: животное	deprel: punct


In [ ]:
# будто все норм, кроме овчарки/Alsatian. Мне кажется, это скорее appos??

#### Задача 4 (3 балла).

Посчитайте частотные списки токенов для каждой категории связей зависимостей (т.е., нужно выделить все токены в тексте, которые получали, например, ярлык amod, и посчитать их частоты). Выведите по первые три самых частотных токена для каждой категории (punct можно не выводить).

In [85]:
def pos_statistics(text):

    if text == eng_text:
        my_nlp = nlp_en
    else:
        my_nlp = nlp_ru

    doc = my_nlp(text)

    d = {} # словарик типа {категория связей зависимости:[токены]}
    for sentence in doc.sentences:
        for word in sentence.words:
            if word.upos != 'PUNCT':
                if word.deprel in d.keys():
                    d[word.deprel].append(word.text)
                else:
                    d[word.deprel] = []
                    d[word.deprel].append(word.text)

    return d

In [86]:
from collections import Counter

In [87]:
eng_stats = pos_statistics(eng_text)

for k, v in eng_stats.items():
    eng_counter = dict(Counter(v))
    eng_tokens = list(eng_counter.keys())
    print(f'{k}: {eng_tokens[0:3]}')

advmod: ['When', 'sleepily', 'only']
nsubj: ['I', 'reaction', 'you']
advcl: ['heard', 'separated', 'overwhelmed']
nmod:poss: ['Laika', 'my', 'your']
case: ["'s", 'in', 'of']
amod: ['frantic', 'first', 'silly']
obj: ['barking', 'fraction', 'eyes']
cop: ['was', 'being', 'be']
root: ['annoyance', 'turned', 'lasted']
compound:prt: ['over', 'up', 'out']
obl: ['bed', 'it', 'moment']
cc: ['and', 'But', 'but']
conj: ['muttered', 'fear', 'more']
parataxis: ['Shut', 'bitch', 'returned']
det: ['a', 'no', 'this']
nmod: ['second', 'loneliness', 'million']
mark: ['of', 'that', 'to']
acl: ['going', 'cause', 'uttering']
xcomp: ['mad', 'open', 'return']
aux: ['did', 'might', 'had']
acl:relcl: ['see', 'comes', 'living']
iobj: ['me', 'myself', 'her']
ccomp: ['set', 'dreaming', 'had']
nsubj:pass: ['Laika', 'that', 'I']
aux:pass: ['was', 'be', 'been']
compound: ['quarter', 'fool', 'Observatory']
obl:agent: ['miles', 'sadness', 'barking']
nummod: ['five', 'one', '95']
fixed: ['course', 'of', 'than']
csubj: 

In [88]:
rus_stats = pos_statistics(rus_text)

for k, v in rus_stats.items():
    rus_counter = dict(Counter(v))
    rus_tokens = list(rus_counter.keys())
    print(f'{k}: {rus_tokens[0:3]}')

amod: ['Неистовый', 'первый', 'другой']
nsubj: ['лай', 'Я', 'дремота']
case: ['в', 'на', 'с']
obl: ['миг', 'бок', 'ним']
advmod: ['только', 'сонно', 'лишь']
root: ['раздосадовал', 'повернулся', 'Замолчи']
obj: ['меня', 'долю', 'глаза']
cc: ['и', 'Но', 'а']
conj: ['буркнул', 'очнулся', 'вернулось']
vocative: ['собака', 'Лайка']
nmod: ['секунды', 'одиночества', 'безумия']
xcomp: ['открыть', 'увидеть', 'уснуть']
iobj: ['мне', 'Тебе', 'себе']
mark: ['что', 'когда', 'хотя']
nummod: ['одна', 'миллиона', 'несколько']
ccomp: ['ступала', 'устоял', 'нет']
det: ['этого', 'своими', 'тот']
parataxis: ['четверть', 'сказал', 'Разумеется']
nummod:gov: ['пять', 'сколько', 'две']
cop: ['будь', 'было', 'был']
acl:relcl: ['увидишь', 'овладевает', 'провел']
nsubj:pass: ['дверь', 'сон', 'животные']
advcl: ['сменяется', 'хотелось', 'ища']
aux:pass: ['был', 'было']
fixed: ['и', 'конце', 'концов']
csubj: ['установить', 'бросить', 'провести']
discourse: ['то']
appos: ['Лайка', 'Андерсон', 'Фарсайда']
compound: 